<a href="https://colab.research.google.com/github/zhaoxiaoer/ml/blob/main/src/2021/hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download Data

Download data from google drive, then unzip it.

You should have `timit_11/train_11.npy`, `timit_11/train_label_11.npy`, and `timit_11/test_11.npy` after running this block.<br><br>

`timit_11/`
- `train_11.npy`: training data<br>
- `train_label_11.npy`: training label<br>
- `test_11.npy`: testing data<br><br>

In [1]:
!ls
%cd /content/drive/MyDrive/Colab\ Notebooks/hw2
!ls

drive  sample_data
/content/drive/MyDrive/Colab Notebooks/hw2
data.zip  hw2.ipynb  model.ckpt  prediction.csv  timit_11


In [ ]:
!gdown --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip
!unzip data.zip
!ls

## Preparing Data

Load the training and testing data from the `.npy` file (NumPy array).

In [2]:
import numpy as np

print('Loading data ...')

data_root = './timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of training label data: {}'.format(train_label.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of training label data: (1229932,)
Size of testing data: (451552, 429)


## Create Dataset

In [3]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
  def __init__(self, X, y=None):
    self.data = torch.from_numpy(X).float()
    if y is not None:
      y = y.astype(np.int)
      self.label = torch.LongTensor(y)
    else:
      self.label = None
  
  def __getitem__(self, idx):
    if self.label is not None:
      return self.data[idx], self.label[idx]
    else:
      return self.data[idx]
  
  def __len__(self):
    return len(self.data)

Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [4]:
VAL_RATIO = 0.2

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (983945, 429)
Size of validation set: (245987, 429)


Create a data loader from the dataset, feel free to tweak the variable `BATCH_SIZE` here.

In [5]:
BATCH_SIZE = 64

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)  # only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

Cleanup the unneeded variables to save memory.

In [6]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

196

## Create Model

Define model architecture, you are encouraged to change and experiment with the model architecture.

In [7]:
import torch
import torch.nn as nn

class Classifier(nn.Module):
  def __init__(self):
    super(Classifier, self).__init__()
    self.layer1 = nn.Linear(429, 256)
    self.layer2 = nn.Linear(256, 256)
    self.layer3 = nn.Linear(256, 128)
    self.out = nn.Linear(128, 39)

    self.act_fn = nn.Sigmoid()

  def forward(self, x):
    x = self.layer1(x)
    x = self.act_fn(x)

    x = self.layer2(x)
    x = self.act_fn(x)

    x = self.layer3(x)
    x = self.act_fn(x)

    x = self.out(x)

    return x

## Training

In [8]:
# check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

Fix random seeds for reproducibility.

In [9]:
def same_seeds(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
  np.random.seed(seed)
  torch.backends.cudnn.benchmark = False
  torch.backends.cudnn.deterministic = True

Feel free to change the training parameters here.

In [10]:
# fix random seed for reproducibility
same_seeds(0)

# get device
device = get_device()
print(f'Device: {device}')

# training parameters
num_epoch = 20          # number of training epoch
learning_rate = 0.0001  # learning rate

# the path where checkpoint saved
model_path = './model.ckpt'

# create model, define a loss function
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Device: cuda


In [12]:
# start training
best_acc = 0.0
for epoch in range(num_epoch):
  train_acc = 0.0
  train_loss = 0.0
  val_acc = 0.0
  val_loss = 0.0

  # training
  model.train()  # set the model to training mode
  for i, data in enumerate(train_loader):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(inputs)
    batch_loss = criterion(outputs, labels)
    _, train_pred = torch.max(outputs, 1)  # get the index of the class with the highest probability
    batch_loss.backward()
    optimizer.step()

    train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
    train_loss += batch_loss.item()
  
  # validation
  if len(val_set) > 0:
    model.eval()  # set the model to evaluation mode
    with torch.no_grad():
      for i, data in enumerate(val_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        batch_loss = criterion(outputs, labels)
        _, val_pred = torch.max(outputs, 1)

        val_acc += (val_pred.cpu() == labels.cpu()).sum().item()
        val_loss += batch_loss.item()

      print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} Loss: {:3.6f}'.format(
          epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
      ))

      # if the model improves, save a checkpoint at this epoch
      if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), model_path)
        print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
      
  else:
    print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
        epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
    ))

# if not validating, save the last epoch
if len(val_set) == 0:
  torch.save(model.state_dict(), model_path)
  print('saving model at last epoch')


[001/020] Train Acc: 0.703412 Loss: 0.924920 | Val Acc: 0.688093 Loss: 0.974711
saving model with acc 0.688
[002/020] Train Acc: 0.705346 Loss: 0.916892 | Val Acc: 0.689740 Loss: 0.967999
saving model with acc 0.690
[003/020] Train Acc: 0.707399 Loss: 0.909380 | Val Acc: 0.691097 Loss: 0.963424
saving model with acc 0.691
[004/020] Train Acc: 0.709388 Loss: 0.902450 | Val Acc: 0.692130 Loss: 0.960527
saving model with acc 0.692
[005/020] Train Acc: 0.711562 Loss: 0.895545 | Val Acc: 0.693313 Loss: 0.955606
saving model with acc 0.693
[006/020] Train Acc: 0.713093 Loss: 0.889330 | Val Acc: 0.694935 Loss: 0.950338
saving model with acc 0.695
[007/020] Train Acc: 0.714841 Loss: 0.883140 | Val Acc: 0.694585 Loss: 0.949040
[008/020] Train Acc: 0.716455 Loss: 0.877309 | Val Acc: 0.695533 Loss: 0.946436
saving model with acc 0.696
[009/020] Train Acc: 0.717776 Loss: 0.872028 | Val Acc: 0.696854 Loss: 0.941757
saving model with acc 0.697
[010/020] Train Acc: 0.719231 Loss: 0.866459 | Val Acc: 

## Testing

Create a testing dataset, and load model from the saved checkpoint.

In [13]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction

In [14]:
predict = []
model.eval()  # set the model to evaluation mode
with torch.no_grad():
  for i, data in enumerate(test_loader):
    inputs = data
    inputs = inputs.to(device)
    outputs = model(inputs)
    _, test_pred = torch.max(outputs, 1)

    for y in test_pred.cpu().numpy():
      predict.append(y)

Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [15]:
with open('prediction.csv', 'w') as f:
  f.write('Id,Class\n')
  for i, y in enumerate(predict):
    f.write('{},{}\n'.format(i, y))